# M3_RAG API Integration Tests

Tests API connectivity for the multi-agent RAG system.

## Models Configuration

| Provider | Generation Model | Embedding Model | Role |
|----------|-----------------|-----------------|------|
| Moonshot/Kimi | moonshot-v1-128k | - | Orchestrator (128k context) |
| Gemini | gemini-2.5-flash | text-embedding-004 | Fast tasks, backup embeddings |
| OpenAI | gpt-4o, o1 | text-embedding-3-large | Reasoning, primary embeddings |
| Anthropic | claude-sonnet-4 | - | Coding, clarity critic |

In [ ]:
# Environment Setup - Load API keys from .env file
import os
import requests
import json
from dotenv import load_dotenv

# Load keys from keyholder.env
load_dotenv('keyholder.env')

MOONSHOT_API_KEY = os.getenv('MOONSHOT_API_KEY')
GEMINI_API_KEY = os.getenv('GEMINI_API_KEY')
OPENAI_API_KEY = os.getenv('OPENAI_API_KEY')
ANTHROPIC_API_KEY = os.getenv('ANTHROPIC_API_KEY')

# Verify keys loaded
keys_status = {
    'MOONSHOT': bool(MOONSHOT_API_KEY),
    'GEMINI': bool(GEMINI_API_KEY),
    'OPENAI': bool(OPENAI_API_KEY),
    'ANTHROPIC': bool(ANTHROPIC_API_KEY)
}
print('API Keys Loaded:')
for name, loaded in keys_status.items():
    status = 'OK' if loaded else 'MISSING'
    print(f'  {name}: {status}')

## 1. Moonshot/Kimi API (Orchestrator)

In [3]:
# Test Moonshot/Kimi API - moonshot-v1-128k
# Role: Orchestrator agent with 128k context window

api_url = "https://api.moonshot.ai/v1/chat/completions"

headers = {
    "Authorization": f"Bearer {MOONSHOT_API_KEY}",
    "Content-Type": "application/json"
}

payload = {
    "model": "moonshot-v1-128k",
    "messages": [{"role": "user", "content": "Reply with: API working"}],
    "temperature": 0.7
}

try:
    print("Testing Moonshot API (moonshot-v1-128k)...")
    response = requests.post(api_url, headers=headers, json=payload, timeout=30)
    response.raise_for_status()
    result = response.json()
    content = result.get('choices', [{}])[0].get('message', {}).get('content', 'No content')
    print(f"Status: OK")
    print(f"Response: {content}")
except requests.exceptions.RequestException as e:
    print(f"Status: FAILED")
    print(f"Error: {e}")

NameError: name 'MOONSHOT_API_KEY' is not defined

## 2. Gemini API (Fast Tasks + Backup Embeddings)

In [ ]:
# Test Gemini API - gemini-2.5-flash
# Role: Problem Definition agent, fast processing tasks

api_url = f"https://generativelanguage.googleapis.com/v1beta/models/gemini-2.5-flash:generateContent?key={GEMINI_API_KEY}"

headers = {"Content-Type": "application/json"}

payload = {
    "contents": [{"parts": [{"text": "Reply with: API working"}]}]
}

try:
    print("Testing Gemini API (gemini-2.5-flash)...")
    response = requests.post(api_url, headers=headers, json=payload, timeout=30)
    response.raise_for_status()
    result = response.json()
    content = result.get('candidates', [{}])[0].get('content', {}).get('parts', [{}])[0].get('text', 'No content')
    print(f"Status: OK")
    print(f"Response: {content}")
except requests.exceptions.RequestException as e:
    print(f"Status: FAILED")
    print(f"Error: {e}")

In [ ]:
# Test Gemini Embeddings - text-embedding-004
# Role: Backup embedding model (768 dimensions)

api_url = f"https://generativelanguage.googleapis.com/v1beta/models/text-embedding-004:embedContent?key={GEMINI_API_KEY}"

headers = {"Content-Type": "application/json"}

payload = {
    "model": "models/text-embedding-004",
    "content": {"parts": [{"text": "Test embedding for M3 mathematical modeling competition"}]}
}

try:
    print("Testing Gemini Embeddings (text-embedding-004)...")
    response = requests.post(api_url, headers=headers, json=payload, timeout=30)
    response.raise_for_status()
    result = response.json()
    embedding = result.get('embedding', {}).get('values', [])
    print(f"Status: OK")
    print(f"Dimensions: {len(embedding)}")
    print(f"Sample values: {embedding[:5]}")
except requests.exceptions.RequestException as e:
    print(f"Status: FAILED")
    print(f"Error: {e}")

## 3. OpenAI API (Reasoning + Primary Embeddings)

In [ ]:
# Test OpenAI API - gpt-4o
# Role: Assumptions/RAG agent, Sensitivity Analysis agent

api_url = "https://api.openai.com/v1/chat/completions"

headers = {
    "Authorization": f"Bearer {OPENAI_API_KEY}",
    "Content-Type": "application/json"
}

payload = {
    "model": "gpt-4o",
    "messages": [{"role": "user", "content": "Reply with: API working"}],
    "temperature": 0.7
}

try:
    print("Testing OpenAI API (gpt-4o)...")
    response = requests.post(api_url, headers=headers, json=payload, timeout=30)
    response.raise_for_status()
    result = response.json()
    content = result.get('choices', [{}])[0].get('message', {}).get('content', 'No content')
    print(f"Status: OK")
    print(f"Response: {content}")
except requests.exceptions.RequestException as e:
    print(f"Status: FAILED")
    print(f"Error: {e}")

In [4]:
# Test OpenAI API - o1 (Reasoning Model)
# Role: Mathematician agent - deep mathematical reasoning

api_url = "https://api.openai.com/v1/chat/completions"

headers = {
    "Authorization": f"Bearer {OPENAI_API_KEY}",
    "Content-Type": "application/json"
}

# o1 requires max_completion_tokens instead of max_tokens
payload = {
    "model": "o1",
    "messages": [{"role": "user", "content": "Reply with: API working"}],
    "max_completion_tokens": 50
}

try:
    print("Testing OpenAI API (o1 - Reasoning Model)...")
    response = requests.post(api_url, headers=headers, json=payload, timeout=60)
    response.raise_for_status()
    result = response.json()
    content = result.get('choices', [{}])[0].get('message', {}).get('content', 'No content')
    print(f"Status: OK")
    print(f"Response: {content}")
except requests.exceptions.RequestException as e:
    print(f"Status: FAILED")
    print(f"Error: {e}")

NameError: name 'OPENAI_API_KEY' is not defined

In [5]:
# Test OpenAI Embeddings - text-embedding-3-large
# Role: Primary embedding model for RAG (3072 dimensions, best quality)

api_url = "https://api.openai.com/v1/embeddings"

headers = {
    "Authorization": f"Bearer {OPENAI_API_KEY}",
    "Content-Type": "application/json"
}

payload = {
    "model": "text-embedding-3-large",
    "input": "Test embedding for M3 mathematical modeling competition assumptions"
}

try:
    print("Testing OpenAI Embeddings (text-embedding-3-large)...")
    response = requests.post(api_url, headers=headers, json=payload, timeout=30)
    response.raise_for_status()
    result = response.json()
    embedding = result.get('data', [{}])[0].get('embedding', [])
    print(f"Status: OK")
    print(f"Dimensions: {len(embedding)}")
    print(f"Sample values: {embedding[:5]}")
except requests.exceptions.RequestException as e:
    print(f"Status: FAILED")
    print(f"Error: {e}")

NameError: name 'OPENAI_API_KEY' is not defined

## 4. Anthropic/Claude API (Coding + Clarity Critic)

In [ ]:
# Test Claude API - claude-sonnet-4
# Role: Coder agent, Clarity critic

api_url = "https://api.anthropic.com/v1/messages"

headers = {
    "x-api-key": ANTHROPIC_API_KEY,
    "anthropic-version": "2023-06-01",
    "Content-Type": "application/json"
}

payload = {
    "model": "claude-sonnet-4-20250514",
    "max_tokens": 100,
    "messages": [{"role": "user", "content": "Reply with: API working"}]
}

try:
    print("Testing Claude API (claude-sonnet-4)...")
    response = requests.post(api_url, headers=headers, json=payload, timeout=30)
    response.raise_for_status()
    result = response.json()
    content = result.get('content', [{}])[0].get('text', 'No content')
    print(f"Status: OK")
    print(f"Response: {content}")
except requests.exceptions.RequestException as e:
    print(f"Status: FAILED")
    print(f"Error: {e}")

In [ ]:
# Test Claude API - claude-opus-4 (Optional: Higher cost, best reasoning)
# Role: Alternative for Mathematician agent if o1 unavailable

api_url = "https://api.anthropic.com/v1/messages"

headers = {
    "x-api-key": ANTHROPIC_API_KEY,
    "anthropic-version": "2023-06-01",
    "Content-Type": "application/json"
}

payload = {
    "model": "claude-opus-4-20250514",
    "max_tokens": 100,
    "messages": [{"role": "user", "content": "Reply with: API working"}]
}

try:
    print("Testing Claude API (claude-opus-4)...")
    response = requests.post(api_url, headers=headers, json=payload, timeout=30)
    response.raise_for_status()
    result = response.json()
    content = result.get('content', [{}])[0].get('text', 'No content')
    print(f"Status: OK")
    print(f"Response: {content}")
except requests.exceptions.RequestException as e:
    print(f"Status: FAILED")
    print(f"Error: {e}")

## 5. Full System Test Summary

In [ ]:
# Run all tests and print summary

def test_api(name, url, headers, payload, extract_fn, timeout=30):
    """Generic API test function."""
    try:
        response = requests.post(url, headers=headers, json=payload, timeout=timeout)
        response.raise_for_status()
        result = response.json()
        return ('OK', extract_fn(result))
    except Exception as e:
        return ('FAILED', str(e)[:50])

# Define all tests
tests = [
    ('Moonshot moonshot-v1-128k', 
     'https://api.moonshot.ai/v1/chat/completions',
     {'Authorization': f'Bearer {MOONSHOT_API_KEY}', 'Content-Type': 'application/json'},
     {'model': 'moonshot-v1-128k', 'messages': [{'role': 'user', 'content': 'OK'}]},
     lambda r: r.get('choices', [{}])[0].get('message', {}).get('content', '')[:20]),
    
    ('Gemini gemini-2.5-flash',
     f'https://generativelanguage.googleapis.com/v1beta/models/gemini-2.5-flash:generateContent?key={GEMINI_API_KEY}',
     {'Content-Type': 'application/json'},
     {'contents': [{'parts': [{'text': 'OK'}]}]},
     lambda r: r.get('candidates', [{}])[0].get('content', {}).get('parts', [{}])[0].get('text', '')[:20]),
    
    ('Gemini text-embedding-004',
     f'https://generativelanguage.googleapis.com/v1beta/models/text-embedding-004:embedContent?key={GEMINI_API_KEY}',
     {'Content-Type': 'application/json'},
     {'model': 'models/text-embedding-004', 'content': {'parts': [{'text': 'test'}]}},
     lambda r: f"{len(r.get('embedding', {}).get('values', []))} dims"),
    
    ('OpenAI gpt-4o',
     'https://api.openai.com/v1/chat/completions',
     {'Authorization': f'Bearer {OPENAI_API_KEY}', 'Content-Type': 'application/json'},
     {'model': 'gpt-4o', 'messages': [{'role': 'user', 'content': 'OK'}]},
     lambda r: r.get('choices', [{}])[0].get('message', {}).get('content', '')[:20]),
    
    ('OpenAI o1',
     'https://api.openai.com/v1/chat/completions',
     {'Authorization': f'Bearer {OPENAI_API_KEY}', 'Content-Type': 'application/json'},
     {'model': 'o1', 'messages': [{'role': 'user', 'content': 'OK'}], 'max_completion_tokens': 10},
     lambda r: r.get('choices', [{}])[0].get('message', {}).get('content', '')[:20]),
    
    ('OpenAI text-embedding-3-large',
     'https://api.openai.com/v1/embeddings',
     {'Authorization': f'Bearer {OPENAI_API_KEY}', 'Content-Type': 'application/json'},
     {'model': 'text-embedding-3-large', 'input': 'test'},
     lambda r: f"{len(r.get('data', [{}])[0].get('embedding', []))} dims"),
    
    ('Claude claude-sonnet-4',
     'https://api.anthropic.com/v1/messages',
     {'x-api-key': ANTHROPIC_API_KEY, 'anthropic-version': '2023-06-01', 'Content-Type': 'application/json'},
     {'model': 'claude-sonnet-4-20250514', 'max_tokens': 10, 'messages': [{'role': 'user', 'content': 'OK'}]},
     lambda r: r.get('content', [{}])[0].get('text', '')[:20]),
    
    ('Claude claude-opus-4',
     'https://api.anthropic.com/v1/messages',
     {'x-api-key': ANTHROPIC_API_KEY, 'anthropic-version': '2023-06-01', 'Content-Type': 'application/json'},
     {'model': 'claude-opus-4-20250514', 'max_tokens': 10, 'messages': [{'role': 'user', 'content': 'OK'}]},
     lambda r: r.get('content', [{}])[0].get('text', '')[:20]),
]

print('=' * 60)
print('M3_RAG API TEST SUMMARY')
print('=' * 60)

for name, url, headers, payload, extract_fn in tests:
    status, info = test_api(name, url, headers, payload, extract_fn, timeout=60)
    print(f'{name:35} {status:8} {info}')

print('=' * 60)